In [43]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd

# Import de la base de données
Le fichier Top_1000_IMDb_movies_New_version.csv contient des informations sur les 1000 meilleurs films selon IMDb.
Il contient les colonnes suivantes :
- `Movie Name` : nom du film.
- `Year of Release` : année de sortie du film.
- `Movie Rating` : Note IMDb du film (sur 10).
- `Metascore of movie` (optionnel) : Score Metacritic du film.
- `Gross` (optionnel) : Revenus générés par le film (en millions de dollars).
- `Votes` : Nombre total de votes reçus sur IMDb.
- `Watch Time` (optionnel) : Durée du film (en minutes).
- `Description` (optionnel) : Résumé ou synopsis du film.
- `Genre` (optionnel) : Genre du film.

In [45]:
df = pd.read_csv("Top_1000_IMDb_movies_New_version.csv")
df.rename(columns={
    "Year of Release": "Released_Year",
    "Movie Rating": "IMDB_Rating",
    "Metascore of movie": "Metascore",
    "Movie Name": "Title"
}, inplace=True) # On change le nom de certaines colonnes en noms plus pratiques

# Nettoyage de données

In [47]:
# Formatage des données numériques
df["Released_Year"] = pd.to_numeric(df["Released_Year"], errors="coerce")
df["Gross"] = pd.to_numeric(df["Gross"].str.replace(",", ""), errors="coerce")
df["Votes"] = pd.to_numeric(df["Votes"].str.replace(",", ""), errors="coerce")

In [49]:
# Suppression des lignes contenant des valeurs nulles pour les colonnes 'Released_Year' et 'IMDB_Rating'
df.dropna(subset=["Released_Year", "IMDB_Rating"], inplace=True)

In [51]:
# Colonnes utiles pour le dashboard

# L'utilisateur pourra visualiser les données par décennies pour des questions de lisibilité
df["Decade"] = (df["Released_Year"] // 10 * 10).astype(int)

# On classe les films en catégories basées sur le nombre de votes qu'ils ont reçus
df["Vote_Range"] = pd.cut(df["Votes"], bins=[0, 500000, 1000000, 2000000, 5000000], labels=["0-500k", "500k-1M", "1M-2M", "2M+"])

# On classe les films en catégories basée sur leur note IMDB
df["Rating_Range"] = pd.cut(df["IMDB_Rating"], bins=[0, 6, 7, 8, 9, 10], labels=["0-6", "6-7", "7-8", "8-9", "9-10"])

# Application Dash
## Interactions utilisateur

Les graphiques s'afficheront avec une variation dépendant de la sélection de l'utilisateur des années sur le *year_slider*.

### Graphiques
- `rating-year-graph` : évolution des notes IMDb par année.
- `gross-votes-scatter` : dispersion des revenus par rapport aux votes.
- `watch-time-histogram` : distribution des durées des films.
- `boxplot-decade` : notes IMDb par décennie.
- `heatmap-duration` : popularité des films par durée et années.

In [57]:
app = dash.Dash(__name__)

# Disposition des éléments
app.layout = html.Div([
    html.Div([
        html.H1("Tableau de Bord IMDb", style={
            "font-size": "2.5em",
            "color": "#333",
            "text-align": "center",
            "margin-bottom": "20px"
        }),
        html.Div(
            dcc.RangeSlider(
                id="year-slider",
                min=int(df["Released_Year"].min()),
                max=int(df["Released_Year"].max()),
                step=1,
                marks={int(year): str(int(year)) for year in df["Released_Year"].unique() if int(year) % 10 == 0},
                value=[int(df["Released_Year"].min()), int(df["Released_Year"].max())]
            ),
            style={"margin-top": "10px","background-color": "#fff",
        "padding": "20px",
        "border-radius": "10px",
        "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
        "margin-bottom": "20px",
        "width": "90%",
        "text-align": "center"}
        )
    ], style={
        # "background-color": "#fff",
        "padding": "20px",
        # "border-radius": "10px",
        # "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
        "margin-bottom": "20px",
        "width": "100%",
        "text-align": "center"
    }),

    html.Div([
        dcc.Graph(id="rating-year-graph", style={
            "background": "white",
            "border-radius": "10px",
            "padding": "10px",
            "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
            "margin": "10px"
        }),
        dcc.Graph(id="gross-votes-scatter", style={
            "background": "white",
            "border-radius": "10px",
            "padding": "10px",
            "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
            "margin": "10px"
        }),
        dcc.Graph(id="watch-time-histogram", style={
            "background": "white",
            "border-radius": "10px",
            "padding": "10px",
            "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
            "margin": "10px"
        }),
        dcc.Graph(id="boxplot-decade", style={
            "background": "white",
            "border-radius": "10px",
            "padding": "10px",
            "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
            "margin": "10px"
        }),
        dcc.Graph(id="heatmap-duration", style={
            "background": "white",
            "border-radius": "10px",
            "padding": "10px",
            "box-shadow": "0px 4px 6px rgba(0, 0, 0, 0.1)",
            "margin": "10px"
        }),
    ], style={
        # "display": "grid",
        # "grid-template-columns": "repeat(auto-fit, minmax(500px, 1fr))",
        "gap": "20px",
        "width": "100%"
    }),
], style={
    "background-color": "#f4f4f9",
    "font-family": "Arial, sans-serif",
    "margin": "0",
    "padding": "20px"
})

@app.callback(
    [Output("rating-year-graph", "figure"),
     Output("gross-votes-scatter", "figure"),
     Output("watch-time-histogram", "figure"),
     Output("boxplot-decade", "figure"),
     Output("heatmap-duration", "figure")],
    [Input("year-slider", "value")]
)
def update_graphs(year_range):
    # Filtrage des données selon l'intervalle d'années
    filtered_df = df[(df["Released_Year"] >= year_range[0]) & (df["Released_Year"] <= year_range[1])]

    # Filtrage des données pour le graphique de dispersion
    scatter_df = filtered_df.dropna(subset=["Gross", "Votes"])

    # Évolution des notes IMDb
    fig1 = px.scatter(filtered_df, x="Released_Year", y="IMDB_Rating",
                      title="Évolution des notes IMDb dans le temps",
                      color="IMDB_Rating",
                      labels={"Released_Year": "Année", "IMDB_Rating": "Note IMDb"},
                      hover_data=["Title"],
                      template="plotly_white")

    # Revenus en fonction des votes
    fig2 = px.scatter(scatter_df, x="Votes", y="Gross", color="IMDB_Rating",
                      size="Gross", hover_data=["Title"],
                      title="Revenus vs Votes",
                      labels={"Votes": "Nombre de votes", "Gross": "Revenus (millions)"},
                      template="plotly_white")

    # Durées des films
    fig3 = px.histogram(filtered_df, x="Watch Time",
                        title="Distribution des durées des films",
                        labels={"Watch Time": "Durée (minutes)"},
                        template="plotly_white")

    # Notes IMDb par décennie
    fig4 = px.box(filtered_df, x="Decade", y="IMDB_Rating", color="Decade",
                  title="Répartition des notes IMDb par décennie",
                  labels={"Decade": "Décennie", "IMDB_Rating": "Note IMDb"},
                  template="plotly_white")

    # Popularité par durée et année
    filtered_df["Duration_Range"] = pd.cut(filtered_df["Watch Time"],
                                           bins=[0, 90, 120, 150, 200, 300],
                                           labels=["0-90 min", "90-120 min", "120-150 min", "150-200 min", "200+ min"])
    heatmap_data = filtered_df.groupby(["Released_Year", "Duration_Range"]).size().reset_index(name="Count")
    fig5 = px.density_heatmap(heatmap_data, x="Released_Year", y="Duration_Range", z="Count",
                              title="Popularité des films par année et durée",
                              labels={"Count": "Nombre de films"},
                              template="plotly_white")

    

    return fig1, fig2, fig3, fig4, fig5

if __name__ == "__main__":
    app.run_server(debug=True, port=8051)